In [1]:
type State = {
    keyValues: Map<string,float>
    counter: int
}

let testState = {
    keyValues = Map.ofList ["a", 1.0; "b", 2.0];
    counter = 100;
}

let getKeyValues state = state.keyValues
let getCounter state = state.counter

// when the type of state isnt declared here, the compiler warns (it still compiles and works though):
// the field labels and expected type of this record expression 
// or pattern do not uniquely determine a corresponding record type
// TODO: why? is this .Net Interactive specific or am I doing something wrong?
let mapKeyValues f (state : State) = {state with keyValues = f (getKeyValues state)} 
let mapCounter f (state : State) = {state with counter = f (getCounter state)}

let get state key = 
    let value = Map.find key (getKeyValues state)
    value

let set state key value =
    let newState = mapKeyValues (Map.add key value) state
    newState

let increment state =
    let newState = mapCounter ((+) 1) state
    let newCounter = getCounter newState
    (newCounter, newState)

let test state = 
    let a = get state "a"
    let b = get state "b"
    let state = set state "c" (a+b)
    let (counter, newState) = increment state
    (counter, newState)

test testState

Item1,Item2
101,"{ keyValues = map [(""a"", 1.0); (""b"", 2.0); (""c"", 3.0)] counter = 101 }"


Signatures of the Functions:
* **get**: State -> string -> float
* **set**: State -> string -> float -> State
* **increment**: State -> int * State

Refactor return to be the same as increment:

In [1]:
let get state key =
    let value = Map.find key (getKeyValues state)
    (value, state)

let set state key value =
    let newState = mapKeyValues (Map.add key value) state
    ((),newState)

now all functions return a tuplet of 2, first value is return value (or unit), second one is the (updated) state

In [1]:
let test state = 
    let (a,state1) = get state "a"
    let (b,state2) = get state1 "b"
    let (_,state3) = set state2 "c" (a+b)
    let (counter, state4) = increment state3
    (counter, state4)

test testState

Item1,Item2
101,"{ keyValues = map [(""a"", 1.0); (""b"", 2.0); (""c"", 3.0)] counter = 101 }"


Refactor state to be last argument:

In [1]:
let get key state =
    let value = Map.find key (getKeyValues state)
    (value, state)

let set key value state =
    let newState = mapKeyValues (Map.add key value) state
    ((),newState)

Now our state is always passed as the last parameter

In [1]:
let test state = 
    let (a,state1) = get "a" state
    let (b,state2) = get "b" state1
    let (_,state3) = set "c" (a+b) state2
    let (counter, state4) = increment state3
    (counter, state4)

test testState

Item1,Item2
101,"{ keyValues = map [(""a"", 1.0); (""b"", 2.0); (""c"", 3.0)] counter = 101 }"


Now refactor to two argument functions

incerement4 adds a dummy variable

In [1]:
let set (key, value) state =
    let newState = mapKeyValues (Map.add key value) state
    ((),newState)

let increment _ state =
    let newState = mapCounter ((+) 1) state
    let newCounter = getCounter newState
    (newCounter, newState)

In [1]:
let test state = 
    let (a,state1) = get "a" state
    let (b,state2) = get "b" state1
    let (_,state3) = set ("c", (a+b)) state2
    let (counter, state4) = increment () state3
    (counter, state4)

test testState

Item1,Item2
101,"{ keyValues = map [(""a"", 1.0); (""b"", 2.0); (""c"", 3.0)] counter = 101 }"


Now we want to refactor state to be an explicit function

In [1]:
let get key =
    fun state -> 
        let value = Map.find key (getKeyValues state)
        (value, state)

let set (key, value) =
    fun state -> 
        let newState = mapKeyValues (Map.add key value) state
        ((),newState)

let increment _ =
    fun state ->
        let newState = mapCounter ((+) 1) state
        let newCounter = getCounter newState
        (newCounter, newState)

I can see set4 as a function with 1 argument which then returns another functions which still needs another argument.

In [1]:
let test state =
    let (a,state1) = get "a" state
    let (b,state2) = get "b" state1
    let (_,state3) = set ("c", (a+b)) state2
    let (counter, state4) = increment () state3
    (counter, state4)

test testState

Item1,Item2
101,"{ keyValues = map [(""a"", 1.0); (""b"", 2.0); (""c"", 3.0)] counter = 101 }"


Feeding output into inputs

In [1]:
let bind2 f g arg state =
    let (rf, state2) = f arg state
    let (rg, state3) = g rf state2
    (rg, state3)

let bind3 f g h arg state = 
    let (rf, state2) = f arg state
    let (rg, state3) = g rf state2
    let (rh, state4) = h rf state3
    (rh, state)

You dont get very far with this approach

Instead call f on arg before binding:

In [1]:
let bind mf g state =
    let (rf, state2) = mf state
    let (rg, state3) = g rf state2
    (rg, state3)

Now instead of writing bind3 which calls all 3 functions I can use bind:

In [1]:
let bind3 mf g h =
    bind (bind mf g) h

Some observations:
* bind seems to be limited because first parameters is "already applied"
  * **Not** an isse! first parameter can be introduced at call point
  * Eg: fun x -> bind (f x) h
* Even better
  * First parameter can be braucht in "from anywhere you want"!
  * Eg: fun x -> ... bind (f x) g ...

Introduction of >>= operator:

In [1]:
let (>>=) mf g =
    bind mf g

let bind3 mf g h =
    mf >>= g >>= h // equivalent to (mf >>= g) >>= h

This assumes the explicit function notation

"Binding" get followed by get

*bind (f x) g* or *(f x) >> g*

So.. *(get "a") >>= get*
Will take the value returned by *get "a"* and give it as input to *get...*
But this is not what we want (also it wouldnt compile since get needs a string as argument)
We want to pass a separate argument to the second call to get and not use the *1*, that *get "a"* returns as an input parameter for the second get call. And we want to use the first get call later.

We want *(get "a") >>= (fun a -> get "b")


In [1]:
let test =
    get "a" >>= fun a ->
    get "b" >>= fun b ->
    set ("c", (a+b)) >>=
    increment

// let test =
//     get "a" 
//     >>= fun a ->
//         get "b" 
//         >>= fun b ->
//             set ("c", (a+b)) 
//             >>= increment
test testState

Item1,Item2
101,"{ keyValues = map [(""a"", 1.0); (""b"", 2.0); (""c"", 3.0)] counter = 101 }"


*get "a"* returns a value which is bound to the a-Variable 

*set ("c", (a+b)) returns a unit value, which can be directly bound to increment

Cleaning up functions with no args:

In [1]:
let (>>/) mf mg =
    mf >>= (fun _ -> mg)

let increment =
    fun state ->
        let newState = mapCounter ((+) 1) state
        let newCounter = getCounter newState
        (newCounter, newState)

In [1]:
let test =
    get "a" >>= fun a ->
    get "b" >>= fun b ->
    set ("c", (a+b)) >>/
    increment

test testState

Item1,Item2
101,"{ keyValues = map [(""a"", 1.0); (""b"", 2.0); (""c"", 3.0)] counter = 101 }"


A first example of "monadic programming":

We write a "monadic" function, which is meant to work with the bind operator

In [1]:
let ret x = fun state -> (x, state)

This will return the value x and just passes the state through

In [1]:
let test =
    get "a" >>= fun a ->
    get "b" >>= fun b ->
    set ("c", (a+b)) >>/
    increment  >>= fun counter ->
    ret counter

test testState

Item1,Item2
101,"{ keyValues = map [(""a"", 1.0); (""b"", 2.0); (""c"", 3.0)] counter = 101 }"


So in the end we get the same result

The "do" notation: computation expressions

First, we need a new type which implements members Bind, Combine and Return

In [1]:
type StateM() =
    member b.Bind (m, f) =
        m >>= f
    member b.Combine (m1, m2) =
        m1 >>/ m2
    member b.Return x =
        ret x

then we can rewrite our test function with the do notation
* *>>=* becomes *let!* ("letBang")
* *>>/* becomes *do!*
* *ret* becomes return

In [1]:
let stateM = StateM()

let test =
    stateM {
        let! a = get "a"
        let! b = get "b"
        do! set ("c", (a+b))
        let! counter = increment
        return counter
    }

test testState

Item1,Item2
101,"{ keyValues = map [(""a"", 1.0); (""b"", 2.0); (""c"", 3.0)] counter = 101 }"


* the bind operator is working with the "lower" expressions first (lazy?)
* if we use the do notation in the lower expression, we get a function, that is compatible with the bind
* it is the same thing as using the earlier implementation
* mixing the two styles is not very common, but it is perfectly ok


In [1]:
let test =
    get "a" >>= fun a ->
    get "b" >>= fun b ->
    stateM {
        do! set ("c", (a+b))
        let! counter = increment
        return counter
    }

test testState

Item1,Item2
101,"{ keyValues = map [(""a"", 1.0); (""b"", 2.0); (""c"", 3.0)] counter = 101 }"


Applying the do notation to all the functions

In [1]:
let getState f = (fun s -> (f s, s))
let mapState f = (fun s -> ((), f s))

let get key =
    stateM {
        let! kv = getState getKeyValues
        return Map.find key kv
    }

let set (key, value) = 
    stateM {
        do! mapState (mapKeyValues (Map.add key value))
    }

let increment = 
    stateM {
        do! mapState (mapCounter ((+) 1))
        let! counter = getState getCounter
        return counter
    }

let test =
    stateM {
        let! a = get "a" 
        let! b = get "b" 
        do! set ("c",(a+b)) 
        let! counter = increment
        return counter
    }

test testState




Item1,Item2
101,"{ keyValues = map [(""a"", 1.0); (""b"", 2.0); (""c"", 3.0)] counter = 101 }"


Welcome to the monad!

A monad is 
* polymorphic - types can be changing from line to line
* composable - compose things together while staying in the same framework of thinking
* flexibly "linkable" with the bind operator
  * "all ready": m\<a\>
  * "waiting for an arg": a->m\<b\>
* carries something hidden - but do I really want that? (debugging?)
* still neds to be executed / run

Making the execution more explicit

In [1]:
let run m state = m state
let test state = run test state
test testState

Item1,Item2
101,"{ keyValues = map [(""a"", 1.0); (""b"", 2.0); (""c"", 3.0)] counter = 101 }"


Giving the monad its own type

In [1]:
type SM<'s,'r> =
    SM of ('s -> 'r*'s)


In [1]:
let getState f =
    SM (fun s -> (f s, s))
let mapState f =
    SM (fun s-> ((), f s))
let ret x =
    SM (fun s -> (x,s))

let run m state =
    match m with
    | SM f -> f state

let bind mf g =
    SM(fun state ->
        let (rf, state2) = run mf state
        let (rg, state3) = run (g rf) state2
        (rg, state3))

bind has 2 arguments which have a monadic character. We need to use run to take them apart. mf is the pure monad, it does not expect any more arguments. g expects 2 arguments and returns a monad when the first argument is applied

In [1]:
type StateM() =
    member b.Bind (m, f) = bind m f
    member b.Combine (m1, m2) = bind m1 (fun _ -> m2)
    member b.Return x = ret x

let stateM = StateM()

let get key =
    stateM {
        let! kv = getState getKeyValues
        return Map.find key kv
    }

let set (key, value) = 
    stateM {
        do! mapState (mapKeyValues (Map.add key value))
    }

let increment = 
    stateM {
        do! mapState (mapCounter ((+) 1))
        let! counter = getState getCounter
        return counter
    }

let test =
    stateM {
        let! a = get "a" 
        let! b = get "b" 
        do! set ("c",(a+b)) 
        let! counter = increment
        return counter
    }

let test1 state = run test state

test1 testState


Item1,Item2
101,"{ keyValues = map [(""a"", 1.0); (""b"", 2.0); (""c"", 3.0)] counter = 101 }"


In [1]:
let bindOpt mf g =
    SM(fun state ->
        let (rf, state2) = run mf state
        match rf with
        | Some rfv -> 
            let (rg, state3) = run (g rfv) state2
            match rg with
            | Some _ -> (rg, state3)
            | None -> (None, state)
        | None -> (None, state))